m8.1stan

m8.1stan is the first model in the Statistical Rethinking book (pp. 249) using Stan.
Here we will use Turing's NUTS support, which is currently (2018) the original NUTS by
Hoffman & Gelman, http://www.stat.columbia.edu/~gelman/research/published/nuts.pdf
and not the one that's in Stan 2.18.2, i.e., Appendix A.5 in:
https://arxiv.org/abs/1701.02434

The StatisticalRethinking pkg uses, e.g., Turing, CSV, DataFrames

In [5]:
using StatisticalRethinking, Turing

d = CSV.read(joinpath(dirname(Base.pathof(StatisticalRethinking)), "..", "data",
    "rugged.csv"), delim=';')
size(d) # Should be 234x51

(234, 51)

Apply log() to each element in rgdppc_2000 column and add it as a new column

In [6]:
d = hcat(d, map(log, d[Symbol("rgdppc_2000")]))
rename!(d, :x1 => :log_gdp) # Rename our col x1 => log_gdp

,isocode,isonum,country,rugged,rugged_popw,rugged_slope,rugged_lsd,rugged_pc,land_area,lat,lon,soil,desert,tropical,dist_coast,near_coast,gemstones,rgdppc_2000,rgdppc_1950_m,rgdppc_1975_m,rgdppc_2000_m,rgdppc_1950_2000_m,q_rule_law,cont_africa,cont_asia,cont_europe,cont_oceania,cont_north_america,cont_south_america,legor_gbr,legor_fra,legor_soc,legor_deu,legor_sca,colony_esp,colony_gbr,colony_fra,colony_prt,colony_oeu,africa_region_n,africa_region_s,africa_region_w,africa_region_e,africa_region_c,slave_exports,dist_slavemkt_atlantic,dist_slavemkt_indian,dist_slavemkt_saharan,dist_slavemkt_redsea,pop_1400,european_descent,log_gdp
,String⍰,Int64⍰,String⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Int64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Int64⍰,Float64⍰,Float64⍰
1,ABW,533,Aruba,0.462,0.38,1.226,0.144,0.0,18.0,12.508,-69.97,21.324,0.0,100.0,0.001,100.0,0,missing,missing,missing,missing,missing,missing,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,missing,missing,missing,missing,614,missing,missing
2,AFG,4,Afghanistan,2.518,1.469,7.414,0.72,39.004,65209.0,33.833,66.026,27.849,4.356,0.0,0.922,0.0,0,missing,644.756,720.633,565.231,679.791,-1.687,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,missing,missing,missing,missing,1870829,0.0,missing
3,AGO,24,Angola,0.858,0.714,2.274,0.228,4.906,124670.0,-12.299,17.551,26.676,0.425,44.346,0.428,13.1587,47756,1794.73,1051.82,1073.04,765.215,1106.76,-1.567,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,3.61e6,5.669,6.981,4.926,3.872,1223208,2.0,7.49261
4,AIA,660,Anguilla,0.013,0.01,0.026,0.006,0.0,9.0,18.231,-63.064,100.0,0.0,100.0,0.0,100.0,0,missing,missing,missing,missing,missing,missing,0,0,0,0,1,0,missing,missing,missing,missing,missing,0,0,0,0,0,0,0,0,0,0,0.0,missing,missing,missing,missing,missing,missing,missing
5,ALB,8,Albania,3.427,1.597,10.451,1.006,62.133,2740.0,41.143,20.07,68.088,0.0,0.0,0.048,94.6919,0,3703.11,1001.34,2289.47,2741.42,1931.78,-0.82,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,missing,missing,missing,missing,200000,100.0,8.21693
6,AND,20,Andorra,5.717,6.722,17.774,1.616,99.064,47.0,42.551,1.576,0.0,0.0,0.0,0.134,0.0,0,missing,missing,missing,missing,missing,1.515,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,missing,missing,missing,missing,missing,missing,missing
7,ANT,530,Netherlands Antilles,0.255,0.039,0.68,0.08,0.0,80.0,12.725,-68.157,24.595,0.0,74.555,0.001,100.0,0,missing,missing,missing,missing,missing,missing,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,missing,missing,missing,missing,1802,missing,missing
8,ARE,784,United Arab Emirates,0.769,0.316,2.112,0.191,6.142,8360.0,23.913,54.331,0.0,77.28,0.0,0.065,75.7464,0,20604.5,15797.6,25465.0,17567.9,20120.0,0.913,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,missing,missing,missing,missing,19200,0.0,9.93326
9,ARG,32,Argentina,0.775,0.22,2.268,0.226,9.407,273669.0,-35.396,-65.17,35.678,0.0,0.0,0.352,13.0167,0,12173.7,4986.73,8122.5,8543.56,6926.81,0.033,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,missing,missing,missing,missing,276632,89.889,9.40703


Now we need to drop every row where rgdppc_2000 == missing
When this (https://github.com/JuliaData/DataFrames.jl/pull/1546) hits DataFrame it'll be
conceptually easier:
i.e., completecases!(d, :rgdppc_2000)

In [7]:
notisnan(e) = !ismissing(e)
dd = d[map(notisnan, d[:rgdppc_2000]), :]
size(dd) # should equal 170 x 52

@model m8_1stan(y, x₁, x₂) = begin
    σ ~ Truncated(Cauchy(0, 2), 0, Inf)
    βR ~ Normal(0, 10)
    βA ~ Normal(0, 10)
    βAR ~ Normal(0, 10)
    α ~ Normal(0, 100)

    for i ∈ 1:length(y)
        y[i] ~ Normal(α + βR * x₁[i] + βA * x₂[i] + βAR * x₁[i] * x₂[i], σ)
    end
end

m8_1stan (generic function with 4 methods)

Test to see that the model is sane. Use 2000 for now, as in the book.
Need to set the same stepsize and adapt_delta as in Stan...

In [8]:
posterior = sample(m8_1stan(dd[:,:log_gdp], dd[:,:rugged], dd[:,:cont_africa]),
    Turing.NUTS(2000, 1000, 0.95));
describe(posterior)

┌ Warning: Indexing with colon as row will create a copy in the future. Use `df[col_inds]` to get the columns without copying
│   caller = top-level scope at In[8]:1
└ @ Core In[8]:1
┌ Warning: Indexing with colon as row will create a copy in the future. Use `df[col_inds]` to get the columns without copying
│   caller = top-level scope at In[8]:1
└ @ Core In[8]:1
┌ Warning: Indexing with colon as row will create a copy in the future. Use `df[col_inds]` to get the columns without copying
│   caller = top-level scope at In[8]:1
└ @ Core In[8]:1
┌ Info: [Turing] looking for good initial eps...
└ @ Turing /Users/rob/.julia/packages/Turing/pRhjG/src/samplers/support/hmc_core.jl:246
[NUTS{Union{}}] found initial ϵ: 0.1
└ @ Turing /Users/rob/.julia/packages/Turing/pRhjG/src/samplers/support/hmc_core.jl:291
[NUTS] Sampling...  1%  ETA: 0:06:53
  ϵ:         0.02565598136606277
  α:         1.0
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling...  2%  ETA: 0:05:29
  ϵ:         0.0194163

4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 40%  ETA: 0:01:12
  ϵ:         0.022838165781990275
  α:         0.992371029879896
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 41%  ETA: 0:01:11
  ϵ:         0.03489339694130155
  α:         0.9848460605610543
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 42%  ETA: 0:01:10
  ϵ:         0.03714758127593374
  α:         0.9663000348161006
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 43%  ETA: 0:01:08
  ϵ:         0.02767680317671709
  α:         0.9978662738235087
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 45%  ETA: 0:01:06
  ϵ:         0.0241601256013915
  α:         0.9775499811620495
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 46%  ETA: 0:01:04
  ϵ:         0.03386137874666251
  α:         0.9955249469130327
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 47%  ETA: 0:01:03
  ϵ:         0.023262423757759693
  α:         0.98

[NUTS] Sampling... 78%  ETA: 0:00:27
  ϵ:         0.022276802018386044
  α:         0.9381291818279136
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 79%  ETA: 0:00:26
  ϵ:         0.022276802018386044
  α:         0.9632709335301358
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 80%  ETA: 0:00:25
  ϵ:         0.022276802018386044
  α:         0.9563503439653344
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 80%  ETA: 0:00:24
  ϵ:         0.022276802018386044
  α:         0.934969021015043
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 81%  ETA: 0:00:23
  ϵ:         0.022276802018386044
  α:         0.9694813707026377
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 82%  ETA: 0:00:21
  ϵ:         0.022276802018386044
  α:         0.9963647928987722
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 84%  ETA: 0:00:20
  ϵ:         0.022276802018386044
  α:         0.9843877684095966
4m  pre_cond:  [1.0, 1

[NUTS] Finished with
  Running time        = 122.09525940599998;
  #lf / sample        = 0.001;
  #evals / sample     = 49.99;
  pre-cond. metric    = [1.0, 1.0, 1.0, 1.0, 1.0].
Iterations = 1:2000
Thinning interval = 1
Chains = 1
Samples per chain = 2000

Empirical Posterior Estimates:
              Mean           SD        Naive SE       MCSE         ESS    
       α    9.156117339  0.647252595 0.0144730080 0.0623855734  107.641381
  lf_num    0.001000000  0.044721360 0.0010000000 0.0010000000 2000.000000
      βA   -1.894712309  0.461920527 0.0103288570 0.0474152885   94.906887
      βR   -0.180344673  0.203144084 0.0045424398 0.0193220597  110.535433
       σ    0.978106285  0.319683724 0.0071483454 0.0287254077  123.853604
 elapsed    0.061047630  0.081078295 0.0018129658 0.0023595384 1180.742247
 epsilon    0.026461600  0.017349097 0.0003879376 0.0010267359  285.519805
eval_num   49.990000000 29.182832605 0.6525479748 1.2838605433  516.677065
     βAR    0.378749126  0.177608706 

Output reg. params of interest:
       Mean           SD        Naive SE        MCSE         ESS
α    9.2140454953  0.416410339 0.00931121825 0.0303436655  188.324543
βA  -1.9414588557  0.373885658 0.00836033746 0.0583949856   40.994586
βR  -0.1987645549  0.158902372 0.00355316505 0.0128657961  152.541295
σ    0.9722532977  0.440031013 0.00983939257 0.0203736871  466.473854
βAR  0.3951414223  0.187780491 0.00419889943 0.0276680621   46.062071

Here's the map2stan output:
       Mean StdDev lower 0.89 upper 0.89 n_eff Rhat
a      9.24   0.14       9.03       9.47   291    1
bR    -0.21   0.08      -0.32      -0.07   306    1
bA    -1.97   0.23      -2.31      -1.58   351    1
bAR    0.40   0.13       0.20       0.63   350    1
sigma  0.95   0.05       0.86       1.03   566    1

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*